<a href="https://colab.research.google.com/github/tedyadika/Azureml/blob/main/3_Data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import Azure ML SDK modules

In [3]:
from azureml.core import Workspace, Datastore,Dataset,Experiment,Run

### Import python modules

In [4]:
%matplotlib inline
import datetime
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date,timedelta
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
pd.set_option('display.max_columns', None)

In [26]:
data_read ='COUNTRY_PICKS_DATA'
filter_column ='ORDTYP_map'
filter_value = "ECOMM"
col_lag_list = ['NBR_ORDERS','SUM_QTY_TOTAL_PICKS','SUM_QTY_PIECE_PICKS']
split_column= 'COUNTRY_NAME'
target_column = 'SUM_QTY_PIECE_PICKS'
cols_drop = ['SUM_QTY_PIECE_PICKS','NBR_ORDERS','SUM_QTY_TOTAL_PICKS']
user_name = 'ADIKA'
experiment_name = ''
threshold= 0.1
name =''

### Set up and data import 

In [10]:
ws = Workspace.get(name= 'dscglobalceedamlt',
                    subscription_id= '8b20e38c-72fb-4030-a702-128e713479ca',
                    resource_group= 'DSCGlobalCEEDTANLTst')

az_store= Datastore.get(ws,'dscglobalceedamlsat_datastore')
az_dataset= Dataset.get_by_name(ws,data_read)
az_default_store = ws.get_default_datastore()
dataset =az_dataset.to_pandas_dataframe()

Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'


### Filter the dataset

In [18]:
df = dataset.copy()
if filter_column and filter_value !=None:
    df = df.query(f'{filter_column}  == "{filter_value}"')
elif filter_column and filter_value == None:
    output_log = 'NO_FILTER APPLIED'
    experiment_run.log(output_log)

### spliting our data by country 

In [12]:
if split_column !=None:
    df_list = []
    for name, group in df.groupby(split_column):
      df_list.append(group)
elif split_column ==None:
    print('Data wont be split')
    df_list.append(df)



### Adding importang parameters to our data

In [13]:
def add_values(data):
    #data = df.copy()
    data['DATE'] = pd.to_datetime(data['DATE'],format ='%Y-%m-%d', errors='coerce')
    data.sort_values('DATE', inplace=True)
    data['Month'] = [i.month for i in data['DATE']]
    data['Year'] = [i.year for i in data['DATE']]
    data['WEEK_YEAR'] = data['DATE'].dt.isocalendar().week 
    data['day_of_week'] = data['DATE'].dt.dayofweek
    data['DAY_YEAR'] = data['DATE'].dt.dayofyear
    data['DATE_N'] = data['DATE'].apply(lambda x:x.toordinal())
    data["WEEK_YEAR"] = data["WEEK_YEAR"].astype('int64').astype('int64')
    return(data)



### Adding Lags

In [14]:
for data in df_list:
  data.sort_values('DATE', inplace=True)
  data['logarithm_base2'] = np.log10(data[target_column])
  data['MA14'] = data[target_column].shift(14).rolling(7).mean()
  data['MA7'] = data[target_column].shift(7).rolling(14).mean()


In [22]:
lag = [7,14,21,28,35]
def add_lags(df_list,column_list,lag_list):
  for i in range(len(df_list)):
      df = df_list[i]#for df in df_list:
      df = df.dropna(subset=['DATE'])
      add_values(df)
      for col in col_lag_list:
        for l in lag:
          new_col_name = col + '_lag_' + str(l)
          new_col_name2 = col + '_mean_' + str(l)
          df_list[i][new_col_name] = df_list[i][col].shift(l)
          df_list[i][new_col_name2] = df_list[i][col].shift(l).rolling(7).mean()
          #df_list[i] = df_list[i].fillna(df.mean())


add_lags(df_list,col_lag_list,lag)


### Resampling data and droping columns 
in this function we resample our data on weekly basis  add date values, drop unwanted columns ,
drop columns with very low correlation and fill nan with mean 

In [23]:
keep_columns =['DATE','Month','Year','WEEK_YEAR','day_of_week','DAY_YEAR','DATE_N']

In [24]:
def drop_columns(dataframe, keep_columns, threshold):
    for column in dataframe.columns:
        if column not in keep_columns and dataframe[column].dtype != 'object':
            corr = dataframe[column].corr(dataframe[target_column])
            if abs(corr) < threshold:
                dataframe = dataframe.drop(column, axis=1)
    return dataframe


In [27]:
if split_column != None:
  for i in range(len(df_list)):
    ml_data = df_list[i].copy()
    drop_columns(ml_data, keep_columns, threshold)
    name = ml_data[split_column].unique()
    ml_data = ml_data.drop(cols_drop, axis=1)
    ml_data.reset_index(inplace = True)
    ml_data[split_column] = name[0]
    ml_data=ml_data.mask(ml_data==0).fillna(ml_data.mean())

    try:
        Dataset.Tabular.register_pandas_dataframe(dataframe = ml_data,
                                               target = az_store,
                                               name = name[0]+'PRED_DATA_'+ experiment_name + user_name,
                                                description=name[0]+'READY FOR ML')
    except Exception as ex:
        print('Error register_pandas_dataframe: ', ex)
elif split_column == None:
   for i in range(len(df_list)):
    ml_data = df_list[i].copy()
    drop_columns(ml_data, keep_columns, threshold)
    ml_data = ml_data.drop(cols_drop, axis=1)
    ml_data.reset_index(inplace = True)
    ml_data[split_column] = name[0]
    ml_data=ml_data.mask(ml_data==0).fillna(ml_data.mean())

    try:
        Dataset.Tabular.register_pandas_dataframe(dataframe = ml_data,
                                               target = az_store,
                                               name = name+'PREP_DATA_'+ experiment_name + user_name,
                                                description=name[0]+'READY FOR ML')
    except Exception as ex:
        print('Error register_pandas_dataframe: ', ex)

<ipython-input-27-59f0836079b1>:9: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  ml_data=ml_data.mask(ml_data==0).fillna(ml_data.mean())
<ipython-input-27-59f0836079b1>:9: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  ml_data=ml_data.mask(ml_data==0).fillna(ml_data.mean())
Message: rslex failed, falling back to clex.
Payload: {"pid": 130, "source": "azureml.dataprep", "version": "", "trace": "azureml|data|dataset_factory.py, line 655 in function register_pandas_dataframe.\nazureml|data|_loggerfactory.py, line 132 in function wrapper.\n<ipython-input-27-59f0836079b1>, line 12 in function <module>.", "subscription": "", "run_id": "", "resource_group": "", "workspace_name": "", "experiment_id": "", "location": "", "rslex_ve

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/50cb1500-9df2-4753-ba3f-8a43997dd1c9/
Successfully uploaded file to datastore.
Creating and registering a new dataset.


Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
<ipython-input-27-59f0836079b1>:9: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  ml_data=ml_data.mask(ml_data==0).fillna(ml_data.mean())
<ipython-input-27-59f0836079b1>:9: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  ml_data=ml_data.mask(ml_data==0).fillna(ml_data.mean())
Message: rslex failed, falling back to clex.
Payload: {"pid": 130, "source": "azureml.dataprep", "version": "", "trace": "azureml|data|dataset_factory.py, line 655 in function register_pandas_dataframe.\nazureml|data|_

Successfully created and registered a new dataset.
Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/25a5a726-9ec5-467d-bc9b-80e7ff224a4e/
Successfully uploaded file to datastore.
Creating and registering a new dataset.


Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
<ipython-input-27-59f0836079b1>:9: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  ml_data=ml_data.mask(ml_data==0).fillna(ml_data.mean())
<ipython-input-27-59f0836079b1>:9: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  ml_data=ml_data.mask(ml_data==0).fillna(ml_data.mean())
Message: rslex failed, falling back to clex.
Payload: {"pid": 130, "source": "azureml.dataprep", "version": "", "trace": "azureml|data|dataset_factory.py, line 655 in function register_pandas_dataframe.\nazureml|data|_

...

Successfully created and registered a new dataset.
Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/71e07aa6-4c3b-4e71-975c-428cd4e8ec18/
Successfully uploaded file to datastore.
Creating and registering a new dataset.


Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
<ipython-input-27-59f0836079b1>:9: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  ml_data=ml_data.mask(ml_data==0).fillna(ml_data.mean())
<ipython-input-27-59f0836079b1>:9: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  ml_data=ml_data.mask(ml_data==0).fillna(ml_data.mean())
Message: rslex failed, falling back to clex.
Payload: {"pid": 130, "source": "azureml.dataprep", "version": "", "trace": "azureml|data|dataset_factory.py, line 655 in function register_pandas_dataframe.\nazureml|data|_

Successfully created and registered a new dataset.
Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/5bd1dfd0-fe82-4687-b424-4797ef23062f/
Successfully uploaded file to datastore.
Creating and registering a new dataset.


Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'
Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info'


Successfully created and registered a new dataset.
